In [7]:
import numpy as np

from torch.utils.data import DataLoader
from torchvision import datasets

from tqdm.notebook import tqdm

import torch
import torch.nn as nn
import torchvision.transforms as Trans

import torch.nn.init as init


In [8]:
BATCH_SIZE = 32
NUM_WORKERS = 4

EPOCHS = 10

In [9]:
transform = Trans.ToTensor()


train_dataset = datasets.MNIST(
    root='datasets',
    train=True,
    transform=transform,
    download=True
)

test_dataset = datasets.MNIST(
    root='datasets',
    train=False,
    transform=transform,
    download=True
)

train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=NUM_WORKERS,
)

In [10]:
class MyNet(nn.Module):
    def __init__(self):
        super().__init__()

        self.flatten = nn.Flatten()

        self.fc1 = nn.Sequential(
            nn.Linear(28*28, 100),
            nn.ReLU()
        )

        self.fc2 = nn.Sequential(
            nn.Linear(100, 50),
            nn.ReLU(),
        )

        self.fcRes = nn.Sequential(
            nn.Linear(50, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.fcRes(x)
        return x

CELoss = nn.CrossEntropyLoss()
net = MyNet()

optimizer = torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [11]:
def train(net, optimizer, criterion):
    running_loss = 0
    for images, labels in train_dataloader:
        
        optimizer.zero_grad()

        output = net(images)

        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss
    with torch.no_grad():
        train_loss = running_loss / len(train_dataloader)
    return train_loss


def valid(net, criterion):
    running_loss = 0
    correct_total = 0
    with torch.no_grad():
        for images, labels in test_dataloader:
            output = net(images)

            loss = criterion(output, labels)
            running_loss += loss

            pred = output.max(dim = 1, keepdim=True)[1]
            correct_total += pred.eq(labels.view_as(pred)).sum()
            
        precison = correct_total / len(test_dataloader.dataset)
        valid_loss = running_loss / len(test_dataloader)
        return valid_loss, precison

In [12]:
for epoch in (pbar := tqdm(range(EPOCHS))):
    train_loss = train(net, optimizer, CELoss)
    valid_loss, prec = valid(net, CELoss)

    print(f"[{epoch}] train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")
    pbar.set_description(f"train/valid loss: {train_loss:.4f}/{valid_loss:.4f} prec: {prec:.4f}")

  0%|          | 0/10 [00:00<?, ?it/s]

[0] train/valid loss: 0.7886/0.6037 prec: 0.7652
[1] train/valid loss: 0.5756/0.5579 prec: 0.7778
[2] train/valid loss: 0.5403/0.5426 prec: 0.7810
[3] train/valid loss: 0.5208/0.5458 prec: 0.7806
[4] train/valid loss: 0.5104/0.5438 prec: 0.7804
[5] train/valid loss: 0.5004/0.5269 prec: 0.7843
[6] train/valid loss: 0.4919/0.5349 prec: 0.7816
[7] train/valid loss: 0.4873/0.5231 prec: 0.7863
[8] train/valid loss: 0.4811/0.5487 prec: 0.7803
[9] train/valid loss: 0.4784/0.5407 prec: 0.7822
